In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.GaussNewton(model=model, lr=1, line_search_method="const")
opt = torch_numopt.GaussNewton(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.GaussNewton(model=model, lr=1, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='wolfe')
# opt = torch_numopt.GaussNewton(model=model, lr=1, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='strong-wolfe')
# opt = torch_numopt.GaussNewton(model=model, lr=1, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='goldstein')

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.34807243943214417
epoch:  1, loss: 0.32638901472091675
epoch:  2, loss: 0.313434898853302
epoch:  3, loss: 0.30219224095344543
epoch:  4, loss: 0.2958122193813324
epoch:  5, loss: 0.2904170751571655
epoch:  6, loss: 0.2904170751571655
epoch:  7, loss: 0.2904170751571655
epoch:  8, loss: 0.26946642994880676
epoch:  9, loss: 0.2584090530872345
epoch:  10, loss: 0.2475092113018036
epoch:  11, loss: 0.23671825230121613
epoch:  12, loss: 0.22504892945289612
epoch:  13, loss: 0.22399595379829407
epoch:  14, loss: 0.213359534740448
epoch:  15, loss: 0.20282182097434998
epoch:  16, loss: 0.1921081393957138
epoch:  17, loss: 0.18177932500839233
epoch:  18, loss: 0.17200811207294464
epoch:  19, loss: 0.16250042617321014
epoch:  20, loss: 0.15448755025863647
epoch:  21, loss: 0.14568190276622772
epoch:  22, loss: 0.136959969997406
epoch:  23, loss: 0.1286754608154297
epoch:  24, loss: 0.12040997296571732
epoch:  25, loss: 0.11272601038217545
epoch:  26, loss: 0.105354443192482


In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.882425856018325
Test metrics:  R2 = 0.5240608551267321
